In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
from copy import deepcopy
import sys
import datasets
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('../experiments/')
import os
import pandas as pd
import numpy as np
import sys
from os.path import join
import datasets
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from imodelsx import AugLinearClassifier

# load dset
dset = datasets.load_dataset('rotten_tomatoes')['train']
dset = dset.select(np.random.choice(len(dset), size=100, replace=False))

dset_val = datasets.load_dataset('rotten_tomatoes')['validation']
dset_val = dset_val.select(np.random.choice(
    len(dset_val), size=100, replace=False))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/thinc/compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/thinc/compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
zeroshot_class_dict = {0: 'negative', 1: 'positive'}

m = AugLinearClassifier(
    checkpoint='textattack/distilbert-base-uncased-rotten-tomatoes',
    ngrams=2,
    all_ngrams=True,  # also use lower-order ngrams
    zeroshot_class_dict=zeroshot_class_dict,
)

In [12]:
m.fit(None, ['negative', 'positive'])

initializing model...
set up linear model...
setting up zero-shot linear model...


AttributeError: 'Embedding' object has no attribute 'vocab'

In [11]:
m.cache_linear_coefs(dset_val['text'])

initializing model...
calculating embeddings...


100%|██████████| 100/100 [00:24<00:00,  4.12it/s]


training linear model...
caching linear coefs...


100%|██████████| 6/6 [00:26<00:00,  4.50s/it]


	After caching, coefs_dict_ len 2654


100%|██████████| 5/5 [00:24<00:00,  4.86s/it]

	After caching, coefs_dict_ len 5046


In [12]:
preds = m.predict(dset['text'])
print('acc_train', np.mean(preds == dset['label']))
preds_proba = m.predict_proba(dset['text'])
preds = m.predict(dset_val['text'])
print('acc_val', np.mean(preds == dset_val['label']))

acc_train 0.99
acc_val 0.77


# Finetuned version

In [6]:
m = AugLinearClassifier(
    checkpoint='textattack/distilbert-base-uncased-rotten-tomatoes',
    ngrams=2,
    all_ngrams=True,  # also use lower-order ngrams
)
m.fit(dset['text'], dset['label'])
m.cache_linear_coefs(dset_val['text'])

initializing model...
calculating embeddings...


100%|██████████| 100/100 [00:40<00:00,  2.45it/s]


training linear model...
caching linear coefs...


100%|██████████| 6/6 [00:23<00:00,  3.88s/it]

	After caching, coefs_dict_ len 2654


AugLinearClassifier(all_ngrams=True,
                    checkpoint='textattack/distilbert-base-uncased-rotten-tomatoes',
                    tokenizer_ngrams=<spacy.tokenizer.Tokenizer object at 0x7f3016097d00>)

In [8]:
preds = m.predict(dset['text'])
print('acc_train', np.mean(preds == dset['label']))
preds_proba = m.predict_proba(dset['text'])
preds = m.predict(dset_val['text'])
print('acc_val', np.mean(preds == dset_val['label']))

acc_train 0.99
acc_val 0.51


/home/chansingh/imodelsx/imodelsx/auglinear/auglinear.py:397: UserWarning: Saw an unseen ungram 2541 times. For better performance, call cache_linear_coefs on the test dataset before calling predict.
  warnings.warn(


In [7]:
print('Total ngram coefficients: ', len(m.coefs_dict_))
print('Most positive ngrams')
for k, v in sorted(m.coefs_dict_.items(), key=lambda item: item[1], reverse=True)[:8]:
    print('\t', k, round(v, 2))
print('Most negative ngrams')
for k, v in sorted(m.coefs_dict_.items(), key=lambda item: item[1])[:8]:
    print('\t', k, round(v, 2))

Total ngram coefficients:  2654
Most positive ngrams
	 a compelling 1.01
	 warm and 0.95
	 stunning . 0.94
	 exquisite acting 0.92
	 compelling 0.91
	 tremendous performance 0.91
	 a tremendous 0.86
	 enjoyable , 0.85
Most negative ngrams
	 idiotic and -0.69
	 is idiotic -0.66
	 tiresome . -0.63
	 condescending and -0.6
	 generic , -0.58
	 gags . -0.56
	 , rather -0.55
	 pretentious and -0.55
